In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_suboptions(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    buttons = soup.find_all('button', class_='btn_sopt')
    subopcao_mapping = {}
    for button in buttons:
        subopcao = button['value']
        nome_produto = button.get_text(strip=True)
        subopcao_mapping[subopcao] = nome_produto
    return subopcao_mapping


In [37]:
def scrape_viti_brasil(ano, opcao):
    opcao_dict = {'opt_05': 'Importação', 'opt_06': 'Exportação'}  # Defina opcao_dict dentro da função

    url = f'http://vitibrasil.cnpuv.embrapa.br/index.php?ano={ano}&opcao={opcao}'
    response = requests.get(url)
    
    if response.status_code == 200:
        subopcao_mapping = extract_suboptions(response.content)
        dfs = []
        for subopcao, produto in subopcao_mapping.items():
            sub_url = f'{url}&subopcao={subopcao}'
            sub_response = requests.get(sub_url)
            if sub_response.status_code == 200:
                sub_soup = BeautifulSoup(sub_response.content, 'html.parser')
                table = sub_soup.find('table', class_='tb_dados')
                if table:
                    data = []
                    rows = table.find_all('tr')
                    for row in rows[1:]:
                        cells = row.find_all('td')
                        row_data = [cell.get_text(strip=True) for cell in cells]
                        data.append(row_data)
                    df = pd.DataFrame(data, columns=['paises', 'quantidade', 'valor'])
                    df['Tabela'] = opcao_dict[opcao]  # Use o dicionário para definir a coluna 'Tabela'
                    df['Produto'] = produto
                    df['Ano'] = ano
                    df['Unidade_quantidade'] = 'kg'
                    df['Unidade_valor'] = 'US$'
                    df = df[['Tabela', 'Produto', 'paises', 'Ano', 'quantidade', 'Unidade_quantidade', 'valor', 'Unidade_valor']]
                    dfs.append(df)
                else:
                    print(f'Tabela não encontrada para subopcao {subopcao}.')
            else:
                print(f'Falha ao fazer a solicitação HTTP para subopcao {subopcao}.')
        
        if dfs:
            return pd.concat(dfs, ignore_index=True)
        else:
            return None
    else:
        print('Falha ao fazer a solicitação HTTP para a página principal.')
        return None

# Exemplo de uso da função
ano = 2023
opcao = 'opt_06'
resultado = scrape_viti_brasil(ano, opcao)
if resultado is not None:
    print(resultado)

         Tabela         Produto                           paises   Ano  \
0    Exportação  Vinhos de mesa                      Afeganistão  2023   
1    Exportação  Vinhos de mesa                    África do Sul  2023   
2    Exportação  Vinhos de mesa  Alemanha, República Democrática  2023   
3    Exportação  Vinhos de mesa                           Angola  2023   
4    Exportação  Vinhos de mesa                         Anguilla  2023   
..          ...             ...                              ...   ...   
495  Exportação     Suco de uva                          Turquia  2023   
496  Exportação     Suco de uva                          Uruguai  2023   
497  Exportação     Suco de uva                          Vanuatu  2023   
498  Exportação     Suco de uva                        Venezuela  2023   
499  Exportação     Suco de uva                            Total  2023   

    quantidade Unidade_quantidade       valor Unidade_valor  
0            -                 kg           -    